In [1]:
import numpy as np
import sys, os
from mipego import ParallelBO, BO, ContinuousSpace, OrdinalSpace, \
    NominalSpace
from mipego.GaussianProcess import AutoencoderGaussianProcess, GaussianProcess, PytorchGaussianProcess
from mipego.GaussianProcess.trend import constant_trend


In [2]:
#test with new autoencoder
dim_r = 2  # dimension of the real values
def obj_fun(x):
    x_r = np.array([x['continuous_%d'%i] for i in range(dim_r)])
    x_i = x['ordinal']
    x_d = x['nominal']
    _ = 0 if x_d == 'OK' else 1
    return np.sum(x_r ** 2) + abs(x_i - 10) / 123. + _ * 2

search_space = ContinuousSpace([-5, 5], var_name='continuous') * dim_r + \
    OrdinalSpace([5, 15], var_name='ordinal') + \
    NominalSpace(['OK', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], var_name='nominal')

model = AutoencoderGaussianProcess(levels=search_space.levels, verbose=True, gpu="13", encoding_dim=5)



In [3]:
model.train_encoder(search_space, sample_size=1000)



11/25/2021 13:17:30,30 - [AutoencoderGaussianProcess.INFO] -- Sampling the search space
11/25/2021 13:17:40,497 - [AutoencoderGaussianProcess.INFO] -- Fitting autoencoder with input shape (1000, 11)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_3 (Dense)              (None, 11)                121       
Total params: 356
Trainable params: 356
Non-trainable params

2021-11-25 13:17:40.883050: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 13:17:41.737342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13803 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:8b:00.0, compute capability: 7.5
2021-11-25 13:17:42.317685: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
4/4 [==============================] - 1s 70ms/step - loss: 0.2072 - val_loss: 0.2057
Epoch 2/50
4/4 [==============================] - 0s 15ms/step - loss: 0.2058 - val_loss: 0.2042
Epoch 3/50
4/4 [==============================] - 0s 15ms/step - loss: 0.2044 - val_loss: 0.2029
Epoch 4/50
4/4 [==============================] - 0s 14ms/step - loss: 0.2032 - val_loss: 0.2016
Epoch 5/50
4/4 [==============================] - 0s 14ms/step - loss: 0.2020 - val_loss: 0.2004
Epoch 6/50
4/4 [==============================] - 0s 14ms/step - loss: 0.2009 - val_loss: 0.1992
Epoch 7/50
4/4 [==============================] - 0s 14ms/step - loss: 0.1998 - val_loss: 0.1980
Epoch 8/50
4/4 [==============================] - 0s 13ms/step - loss: 0.1987 - val_loss: 0.1968
Epoch 9/50
4/4 [==============================] - 0s 14ms/step - loss: 0.1976 - val_loss: 0.1956
Epoch 10/50
4/4 [==============================] - 0s 15ms/step - loss: 0.1965 - val_loss: 0.1944
Epoch 11/50
4/4 [============

In [4]:
from sklearn.preprocessing import MinMaxScaler
data = search_space.sampling(10, method='LHS')
#use the search_space.bounds for min/max scaling
data = model._check_X(data)
i = 0
for t in search_space.var_type:
    b = search_space.bounds[i]
    if (t != "N"):
        data[:,i] = (data[:,i] - b[0]) / (b[1] - b[0])
    i+=1

#scaler = MinMaxScaler().fit(data)
#data = scaler.transform(data)
data[0]

array([0.45, 0.65, 0.1 , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ])

In [5]:
#y = model.encoder.predict(data)
#print(y)
y_ = model.autoencoder.predict(data)
print(y_[0])

[0.4870303  0.49366486 0.4184984  0.2820496  0.07239746 0.14645042
 0.37498325 0.12363555 0.09095247 0.1112429  0.22282511]


In [6]:
opt = BO(
            search_space=search_space, 
            obj_fun=obj_fun, 
            model=model, 
            max_FEs=9,
            DoE_size=3,    # the initial DoE size
            eval_type='dict',
            acquisition_fun='EI',
            acquisition_par={'plugin' : 2},
            verbose=True   # turn this off, if you prefer no output
        )
xopt, fopt, stop_dict = opt.run()

print('xopt: {}'.format(xopt))
print('fopt: {}'.format(fopt))
print('stop criteria: {}'.format(stop_dict))

11/25/2021 13:17:46,377 - [BO.INFO] -- evaluation takes 0.0001s
11/25/2021 13:17:46,378 - [BO.INFO] -- initial DoE of size 3:
11/25/2021 13:17:46,379 - [BO.INFO] -- #1 - fitness: 13.135501355013542, solution: {'continuous_0': 3.333333333333332, 'continuous_1': 0.0, 'ordinal': 13, 'nominal': 'D'}
11/25/2021 13:17:46,380 - [BO.INFO] -- #2 - fitness: 24.246612466124652, solution: {'continuous_0': -3.3333333333333335, 'continuous_1': 3.333333333333332, 'ordinal': 7, 'nominal': 'G'}
11/25/2021 13:17:46,381 - [BO.INFO] -- #3 - fitness: 13.119241192411925, solution: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
11/25/2021 13:17:46,382 - [BO.INFO] -- fopt: 13.119241192411925
11/25/2021 13:17:46,383 - [BO.INFO] -- xopt: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
[[0.17911333 1.9508486  0.99210864 0.         0.1396551 ]
 [0.46578884 1.6167525  0.860814   0.         0.09068546]
 [0.7589147  2.5238001  0.79634

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:20:03,233 - [BO.INFO] -- evaluation takes 0.0001s
11/25/2021 13:20:03,235 - [BO.INFO] -- iteration 1, 1 infill points:
11/25/2021 13:20:03,236 - [BO.INFO] -- #1 - fitness: 26.733516443979074, solution: {'continuous_0': 4.9136392313154005, 'continuous_1': 0.7409558299610097, 'ordinal': 15, 'nominal': 'A'}
11/25/2021 13:20:03,237 - [BO.INFO] -- fopt: 13.119241192411925
11/25/2021 13:20:03,237 - [BO.INFO] -- xopt: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
[[1.7911333e-01 1.9508486e+00 9.9210864e-01 0.0000000e+00 1.3965510e-01]
 [4.6578884e-01 1.6167525e+00 8.6081398e-01 0.0000000e+00 9.0685457e-02]
 [7.5891471e-01 2.5238001e+00 7.9634434e-01 0.0000000e+00 4.5131588e-01]
 [1.8853161e+00 2.4670837e+00 2.4873838e-03 0.0000000e+00 5.7587147e-01]]
[13.13550136 24.24661247 13.11924119 26.73351644]
11/25/2021 13:20:03,293 - [AutoencoderGaussianProcess.INFO] -- Training the GPytorch exact inference model
11/25/2021 13:20:03,298 - [Autoe

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:22:23,993 - [BO.INFO] -- evaluation takes 0.0000s
11/25/2021 13:22:23,995 - [BO.INFO] -- iteration 2, 1 infill points:
11/25/2021 13:22:23,996 - [BO.INFO] -- #1 - fitness: 24.12668560542261, solution: {'continuous_0': 0.5773088184650987, 'continuous_1': -4.873679280281067, 'ordinal': 5, 'nominal': 'OK'}
11/25/2021 13:22:24,03 - [BO.INFO] -- fopt: 13.119241192411925
11/25/2021 13:22:24,03 - [BO.INFO] -- xopt: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
[[1.7911333e-01 1.9508486e+00 9.9210864e-01 0.0000000e+00 1.3965510e-01]
 [4.6578884e-01 1.6167525e+00 8.6081398e-01 0.0000000e+00 9.0685457e-02]
 [7.5891471e-01 2.5238001e+00 7.9634434e-01 0.0000000e+00 4.5131588e-01]
 [1.8853161e+00 2.4670837e+00 2.4873838e-03 0.0000000e+00 5.7587147e-01]
 [0.0000000e+00 1.2196068e+00 1.5187572e+00 0.0000000e+00 8.5932839e-01]]
[13.13550136 24.24661247 13.11924119 26.73351644 24.12668561]
11/25/2021 13:22:24,65 - [AutoencoderGaussianProcess.INFO

...

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:29:29,982 - [BO.INFO] -- evaluation takes 0.0000s
11/25/2021 13:29:29,985 - [BO.INFO] -- iteration 5, 1 infill points:
11/25/2021 13:29:29,986 - [BO.INFO] -- #1 - fitness: 27.83113904160688, solution: {'continuous_0': 1.8631742860145106, 'continuous_1': -4.93143693207539, 'ordinal': 5, 'nominal': 'OK'}
11/25/2021 13:29:29,987 - [BO.INFO] -- fopt: 13.119241192411925
11/25/2021 13:29:29,988 - [BO.INFO] -- xopt: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
[[1.7911333e-01 1.9508486e+00 9.9210864e-01 0.0000000e+00 1.3965510e-01]
 [4.6578884e-01 1.6167525e+00 8.6081398e-01 0.0000000e+00 9.0685457e-02]
 [7.5891471e-01 2.5238001e+00 7.9634434e-01 0.0000000e+00 4.5131588e-01]
 [1.8853161e+00 2.4670837e+00 2.4873838e-03 0.0000000e+00 5.7587147e-01]
 [0.0000000e+00 1.2196068e+00 1.5187572e+00 0.0000000e+00 8.5932839e-01]
 [7.8295186e-02 8.3339894e-01 7.0091647e-01 0.0000000e+00 8.6073734e-02]
 [1.9213647e+00 2.4561591e+00 0.0000000e+00 0.

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:31:51,40 - [BO.INFO] -- evaluation takes 0.0000s
11/25/2021 13:31:51,42 - [BO.INFO] -- iteration 6, 1 infill points:
11/25/2021 13:31:51,42 - [BO.INFO] -- #1 - fitness: 28.704285660410314, solution: {'continuous_0': 1.9140625, 'continuous_1': -5.0, 'ordinal': 5, 'nominal': 'OK'}
11/25/2021 13:31:51,43 - [BO.INFO] -- fopt: 13.119241192411925
11/25/2021 13:31:51,44 - [BO.INFO] -- xopt: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 9, 'nominal': 'C'}
[[1.7911333e-01 1.9508486e+00 9.9210864e-01 0.0000000e+00 1.3965510e-01]
 [4.6578884e-01 1.6167525e+00 8.6081398e-01 0.0000000e+00 9.0685457e-02]
 [7.5891471e-01 2.5238001e+00 7.9634434e-01 0.0000000e+00 4.5131588e-01]
 [1.8853161e+00 2.4670837e+00 2.4873838e-03 0.0000000e+00 5.7587147e-01]
 [0.0000000e+00 1.2196068e+00 1.5187572e+00 0.0000000e+00 8.5932839e-01]
 [7.8295186e-02 8.3339894e-01 7.0091647e-01 0.0000000e+00 8.6073734e-02]
 [1.9213647e+00 2.4561591e+00 0.0000000e+00 0.0000000e+00 4.1325349e-01]

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [ ]:
from mipego import ParallelBO, BO, ContinuousSpace, OrdinalSpace, \
    NominalSpace, RandomForest
model = RandomForest(levels=search_space.levels)

opt = ParallelBO(
    search_space=search_space, 
    obj_fun=obj_fun, 
    model=model, 
    max_FEs=9, 
    DoE_size=3,    # the initial DoE size
    eval_type='dict',
    acquisition_fun='MGFI',
    acquisition_par={'t' : 2},
    n_job=3,       # number of processes
    n_point=3,     # number of the candidate solution proposed in each iteration
    verbose=True   # turn this off, if you prefer no output
)
xopt, fopt, stop_dict = opt.run()

print('xopt: {}'.format(xopt))
print('fopt: {}'.format(fopt))
print('stop criteria: {}'.format(stop_dict))

11/25/2021 09:32:43,646 - [ParallelBO.INFO] -- evaluation takes 0.4571s
11/25/2021 09:32:43,648 - [ParallelBO.INFO] -- initial DoE of size 3:
11/25/2021 09:32:43,648 - [ParallelBO.INFO] -- #1 - fitness: 24.246612466124652, solution: {'continuous_0': -3.3333333333333335, 'continuous_1': 3.333333333333332, 'ordinal': 7, 'nominal': 'G'}
11/25/2021 09:32:43,649 - [ParallelBO.INFO] -- #2 - fitness: 13.119241192411925, solution: {'continuous_0': 0.0, 'continuous_1': -3.3333333333333335, 'ordinal': 11, 'nominal': 'E'}
11/25/2021 09:32:43,649 - [ParallelBO.INFO] -- #3 - fitness: 13.119241192411916, solution: {'continuous_0': 3.333333333333332, 'continuous_1': 0.0, 'ordinal': 9, 'nominal': 'F'}
11/25/2021 09:32:43,650 - [ParallelBO.INFO] -- fopt: 13.119241192411916
11/25/2021 09:32:43,651 - [ParallelBO.INFO] -- xopt: {'continuous_0': 3.333333333333332, 'continuous_1': 0.0, 'ordinal': 9, 'nominal': 'F'}
11/25/2021 09:32:43,765 - [ParallelBO.INFO] -- Surrogate model r2: -0.004050000000000109

11/

In [2]:
dim = 5
lb, ub = -1, 5

def fitness(x):
    x = np.asarray(x)
    return np.sum(x ** 2)

space = ContinuousSpace([lb, ub]) * dim
model = PytorchGaussianProcess()

opt = BO(
    search_space=space, 
    obj_fun=fitness, 
    model=model, 
    DoE_size=5,
    max_FEs=10, 
    verbose=True, 
    n_point=1
)
print(opt.run())

11/25/2021 13:04:05,719 - [BO.INFO] -- evaluation takes 0.0001s
11/25/2021 13:04:05,720 - [BO.INFO] -- initial DoE of size 5:
11/25/2021 13:04:05,721 - [BO.INFO] -- #1 - fitness: 56.96000000000001, solution: [4.4, 3.2, 4.4, 2.0, 2.0]
11/25/2021 13:04:05,722 - [BO.INFO] -- #2 - fitness: 41.120000000000005, solution: [3.2, 0.8000000000000003, 0.8000000000000003, 4.4, 3.2]
11/25/2021 13:04:05,723 - [BO.INFO] -- #3 - fitness: 11.840000000000003, solution: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2, 0.8000000000000003]
11/25/2021 13:04:05,724 - [BO.INFO] -- #4 - fitness: 30.560000000000006, solution: [-0.3999999999999999, 4.4, 3.2, 0.8000000000000003, -0.3999999999999999]
11/25/2021 13:04:05,725 - [BO.INFO] -- #5 - fitness: 31.520000000000003, solution: [2.0, 2.0, 2.0, -0.3999999999999999, 4.4]
11/25/2021 13:04:05,725 - [BO.INFO] -- fopt: 11.840000000000003
11/25/2021 13:04:05,726 - [BO.INFO] -- xopt: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:04:23,344 - [BO.INFO] -- evaluation takes 0.0001s
11/25/2021 13:04:23,346 - [BO.INFO] -- iteration 1, 1 infill points:
11/25/2021 13:04:23,347 - [BO.INFO] -- #1 - fitness: 52.9999996846046, solution: [-0.9999999979644469, -0.9999999691042197, 4.999999992228182, 4.999999987506143, -0.9999999765620059]
11/25/2021 13:04:23,348 - [BO.INFO] -- fopt: 11.840000000000003
11/25/2021 13:04:23,349 - [BO.INFO] -- xopt: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2, 0.8000000000000003]
+----+-------+-------+-------+-------+-------+----------+-------+
|    |   r_0 |   r_1 |   r_2 |   r_3 |   r_4 |   n_eval |     f |
+====+=======+=======+=======+=======+=======+==========+=======+
|  0 |   4.4 |   3.2 |   4.4 |   2   |   2   |        1 | 56.96 |
+----+-------+-------+-------+-------+-------+----------+-------+
|  1 |   3.2 |   0.8 |   0.8 |   4.4 |   3.2 |        1 | 41.12 |
+----+-------+-------+-------+-------+-------+----------+-------+
|  2 |   0.8 |  -0.4 |  -

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:04:38,997 - [BO.INFO] -- evaluation takes 0.0000s
11/25/2021 13:04:38,999 - [BO.INFO] -- iteration 2, 1 infill points:
11/25/2021 13:04:39,00 - [BO.INFO] -- #1 - fitness: 28.99993423220995, solution: [4.9999971464597674, -0.9999847276285082, -0.9999977146358969, -0.9999994589091465, -0.9999994825089986]
11/25/2021 13:04:39,01 - [BO.INFO] -- fopt: 11.840000000000003
11/25/2021 13:04:39,01 - [BO.INFO] -- xopt: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2, 0.8000000000000003]
+----+-------+-----------+-----------+-----------+-----------+----------+---------+
|    |   r_0 |       r_1 |       r_2 |       r_3 |       r_4 |   n_eval |       f |
+====+=======+===========+===========+===========+===========+==========+=========+
|  0 |   4.4 |  3.2      |  4.4      |  2        |  2        |        1 | 56.96   |
+----+-------+-----------+-----------+-----------+-----------+----------+---------+
|  1 |   3.2 |  0.8      |  0.8      |  4.4      |  3.2      |    

...

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:05:11,159 - [BO.INFO] -- evaluation takes 0.0000s
11/25/2021 13:05:11,161 - [BO.INFO] -- iteration 4, 1 infill points:
11/25/2021 13:05:11,162 - [BO.INFO] -- #1 - fitness: 31.50075189224937, solution: [1.871033514019437, -0.9999999798851729, 4.999999021900494, -0.9999987602174475, -0.9999988912257716]
11/25/2021 13:05:11,162 - [BO.INFO] -- fopt: 11.840000000000003
11/25/2021 13:05:11,163 - [BO.INFO] -- xopt: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2, 0.8000000000000003]
+----+----------+-----------+-----------+-----------+-----------+----------+---------+
|    |      r_0 |       r_1 |       r_2 |       r_3 |       r_4 |   n_eval |       f |
+====+==========+===========+===========+===========+===========+==========+=========+
|  0 |  4.4     |  3.2      |  4.4      |  2        |  2        |        1 | 56.96   |
+----+----------+-----------+-----------+-----------+-----------+----------+---------+
|  1 |  3.2     |  0.8      |  0.8      |  4.4     

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


11/25/2021 13:05:27,759 - [BO.INFO] -- evaluation takes 0.0001s
11/25/2021 13:05:27,760 - [BO.INFO] -- iteration 5, 1 infill points:
11/25/2021 13:05:27,761 - [BO.INFO] -- #1 - fitness: 49.38349296603974, solution: [4.715879022776705, 4.994796250482041, -0.8910766031976347, -0.9416851447357978, -0.7177743379103205]
11/25/2021 13:05:27,761 - [BO.INFO] -- fopt: 11.840000000000003
11/25/2021 13:05:27,762 - [BO.INFO] -- xopt: [0.8000000000000003, -0.3999999999999999, -0.3999999999999999, 3.2, 0.8000000000000003]
+----+----------+-----------+-----------+-----------+-----------+----------+---------+
|    |      r_0 |       r_1 |       r_2 |       r_3 |       r_4 |   n_eval |       f |
+====+==========+===========+===========+===========+===========+==========+=========+
|  0 |  4.4     |  3.2      |  4.4      |  2        |  2        |        1 | 56.96   |
+----+----------+-----------+-----------+-----------+-----------+----------+---------+
|  1 |  3.2     |  0.8      |  0.8      |  4.4     

/home/neocortex/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
